# Feed-Forward Neural Networks vs Recurrent Neural Networks

A feed-forward neural network allows information to flow only in the forward direction, from the input nodes, through the hidden layers, and to the output nodes. There are no cycles or loops in the network. 

In a feed-forward neural network, the decisions are based on the current input. It doesn’t memorize the past data, and there’s no future scope. Feed-forward neural networks are used in general regression and classification problems.

# **RNN**

Recurrent Neural Networks (RNNs) are a type of artificial neural network designed to process sequences of data. They work especially well for jobs requiring sequences, such as time series data, voice, natural language, and other activities.

RNN works on the principle of saving the output of a particular layer and feeding this back to the input in order to predict the output of the layer.

Below is how you can convert a Feed-Forward Neural Network into a Recurrent Neural Network:

<img src="images/nn2rnn.gif" width=300/>

In [1]:
import numpy as np

In [2]:
class ReccurentNN:
    def __init__(self, char_to_idx, idx_to_char, vocab, h_size=75,
                seq_len=20, clip_value=5, epochs=50, learning_rate=1e-2):
        self.n_h = h_size 
        self.seq_len = seq_len 
        self.clip_value = clip_value  
        self.epochs = epochs  
        self.learning_rate = learning_rate
        self.char_to_idx = char_to_idx  
        self.idx_to_char = idx_to_char  
        self.vocab = vocab  

        self.smooth_loss = -np.log(1.0 / self.vocab) * self.seq_len  

        # -----initialise weights and biases----- #
        self.params = {}
        self.params["W_xh"] = np.random.randn(self.vocab, self.n_h) * 0.01 
        self.params["W_hh"] = np.identity(self.n_h) * 0.01
        self.params["b_h"] = np.zeros((1, self.n_h))
        self.params["W_hy"] = np.random.randn(self.n_h, self.vocab) * 0.01
        self.params["b_y"] = np.zeros((1, self.vocab))
        self.h0 = np.zeros((1, self.n_h))  # value of the hidden state at time step t = -1

        # -----initialise gradients and memory parameters for Adagrad ----- # 
        self.grads = {}
        self.m_params = {}
        for key in self.params:
            self.grads["d" + key] = np.zeros_like(self.params[key])
            self.m_params["m" + key] = np.zeros_like(self.params[key])

    def _encode_text(self, X):
        X_encoded = []
        for char in X:
            X_encoded.append(self.char_to_idx[char])
        return X_encoded

    def _prepare_batches(self, X, index):
            X_batch_encoded = X[index: index + self.seq_len]
            y_batch_encoded = X[index + 1: index + self.seq_len + 1]
            X_batch = []
            y_batch = []
            for i in X_batch_encoded:
                one_hot_char = np.zeros((1, self.vocab))
                one_hot_char[0][i] = 1
                X_batch.append(one_hot_char)
            for j in y_batch_encoded:
                one_hot_char = np.zeros((1, self.vocab))
                one_hot_char[0][j] = 1
                y_batch.append(one_hot_char)
            return X_batch, y_batch 
    
    def _softmax(self, x):
        e_x = np.exp(x - np.max(x)) 
        return e_x / np.sum(e_x)
    
    def _forward_pass(self, X):
        h = {}  
        h[-1] = self.h0  # set initial hidden state at t=-1
        y_pred = {}  # stores softmax output probabilities

        # iterate over each character in the input sequence
        for t in range(self.seq_len):
            h[t] = np.tanh(
                np.dot(X[t], self.params["W_xh"]) + np.dot(h[t - 1], self.params["W_hh"]) + self.params["b_h"])
            y_pred[t] = self._softmax(np.dot(h[t], self.params["W_hy"]) + self.params["b_y"])
        self.ho = h[t]
        return y_pred, h
    
    def _backward_pass(self, X, y, y_pred, h):
        dh_next = np.zeros_like(h[0])

        for t in reversed(range(self.seq_len)):
            dy = np.copy(y_pred[t])
            dy[0][np.argmax(y[t])] -= 1  # predicted y - actual y
            self.grads["dW_hy"] += np.dot(h[t].T, dy)
            self.grads["db_y"] += dy
            dhidden = (1 - h[t] ** 2) * (np.dot(dy, self.params["W_hy"].T) + dh_next)
            dh_next = np.dot(dhidden, self.params["W_hh"].T)
            self.grads["dW_hh"] += np.dot(h[t - 1].T, dhidden)
            self.grads["dW_xh"] += np.dot(X[t].T, dhidden)
            self.grads["db_h"] += dhidden

        for grad, key in enumerate(self.grads):
            np.clip(self.grads[key], -self.clip_value, self.clip_value, out=self.grads[key])
        return
    
    def _update(self):
        for key in self.params:
            self.m_params["m" + key] += self.grads["d" + key] * self.grads["d" + key]
            self.params[key] -= self.grads["d" + key] * self.learning_rate / (np.sqrt(self.m_params["m" + key]) + 1e-8) 
  
    def train(self, X):
        loss = []

        num_batches = len(X) // self.seq_len
        X_trimmed = X[:num_batches * self.seq_len]

        X_encoded = self._encode_text(X_trimmed)

        for i in range(self.epochs):
            for j in range(0, len(X_encoded) - self.seq_len, self.seq_len):
                X_batch, y_batch = self._prepare_batches(X_encoded, j)

                # Forward pass
                y_pred, h = self._forward_pass(X_batch)

                # loss calculation
                loss_batch = 0
                for t in range(self.seq_len):
                    loss_batch += -np.log(y_pred[t][0, np.argmax(y_batch[t])])

                # Update smooth loss and append to loss list
                self.smooth_loss = self.smooth_loss * 0.999 + loss_batch * 0.001
                loss.append(self.smooth_loss)

                # Backward pass and update parameters
                self._backward_pass(X_batch, y_batch, y_pred, h)
                self._update()

            # Print training progress
            print(f'Epoch: {i + 1}\tLoss: {loss[-1]}')
            print(self.test(50, 2))
        return loss, self.params

    def test(self, test_size, start_index):
        res = ""
        x = np.zeros((1, self.vocab))
        x[0][start_index] = 1
        for i in range(test_size):
            # forward propagation
            h = np.tanh(np.dot(x, self.params["W_xh"]) + np.dot(self.h0, self.params["W_hh"]) + self.params["b_h"])
            y_pred = self._softmax(np.dot(h, self.params["W_hy"]) + self.params["b_y"])
            # get a random index from the probability distribution of y
            index = np.random.choice(range(self.vocab), p=y_pred.ravel())
            # set x-one_hot_vector for the next character
            x = np.zeros((1, self.vocab))
            x[0][index] = 1
            # find the char with the index and concat to the output string
            char = self.idx_to_char[index]
            res += char
        return res

In [3]:
with open('HarryPotter_AllBooks.txt') as file:
    text = file.read().lower()

In [4]:
# use only a part of the text to make the process faster
text = text[:50000] 
chars = set(text)
vocab = len(chars)

In [5]:
# creating the encoding decoding dictionaries
char_to_idx = {w: i for i, w in enumerate(chars)}
idx_to_char = {i: w for i, w in enumerate(chars)}
rnnParameters = {
        'epochs': 10,
        'char_to_idx': char_to_idx,
        'idx_to_char': idx_to_char,
        'vocab': vocab,
        'h_size': 75,
        'seq_len': 20,  # keeping small to avoid diminishing/exploding gradients
        'clip_value': 5,
        'learning_rate': 1e-2,
    }

# How Does Recurrent Neural Networks Work?

In Recurrent Neural networks, the information cycles through a loop to the middle hidden layer.

<img src="images/rnn.gif" width=500/>

The input layer ‘x’ takes in the input to the neural network and processes it and passes it onto the middle layer. 

The middle layer ‘h’ can consist of multiple hidden layers, each with its own activation functions and weights and biases. If you have a neural network where the various parameters of different hidden layers are not affected by the previous layer, ie: the neural network does not have memory, then you can use a recurrent neural network.

The Recurrent Neural Network will standardize the different activation functions and weights and biases so that each hidden layer has the same parameters. Then, instead of creating multiple hidden layers, it will create one and loop over it as many times as required. 

In [6]:
RNN = ReccurentNN(**rnnParameters)
rnnLoss, params = RNN.train(text)

Epoch: 1	Loss: 52.32316888461033
hind bs is ptnthorey thert s tr asthord ved s .co 
Epoch: 2	Loss: 48.093867837723565
b aulappe cidy .nren pt wirsatd thikedecaavee hick
Epoch: 3	Loss: 46.8850690476119
pqudnisd lid or a .momole ike tt wrey slt ots ver 
Epoch: 4	Loss: 46.35404033768812
thed .itice onccareming py sf t ane .d budavecsme 
Epoch: 5	Loss: 46.04717780354482
tid lsad ny ls winounon altle t a f sastton tlleck
Epoch: 6	Loss: 45.8411466304117
tomyt f l ines alt s lifided ad ifr at satirry lar
Epoch: 7	Loss: 45.67792053455727
the upe mce heyle ?t cko ambinonomond .heravetheys
Epoch: 8	Loss: 45.56331430772447
t wixkint therang p .he d fut ne sacklit oupevery 
Epoch: 9	Loss: 45.471320088592115
ingndonit .doke cadase .dotconivy fay wraikey was 
Epoch: 10	Loss: 45.40079313050033
wanat s lchedin ade n ladat canale oustonad whel t


> ### The main limitation of RNNs is that RNNs can’t remember very long sequences and get into the problem of vanishing gradient.

# **LSTM**

LSTMs come to the rescue to solve the vanishing gradient problem. It does so by ignoring (forgetting) useless data/information in the network. The LSTM will forget the data if there is no useful information from other inputs (prior sentence words). When new information comes, the network determines which information to be overlooked and which to be remembered.

LSTMs have 4 different components, namely
Cell state (Memory cell)
Forget gate
Input gate
Output gate

<img src="images/lstmArchitecture.jpeg" width=500/>

In [7]:
class rnnLSTM:
    def __init__(self, char_to_idx, idx_to_char, epochs=10, vocab=20, n_h=100, 
                 seq_len=25, learning_rate=0.001, beta1=0.9, beta2=0.999):
        self.char_to_idx = char_to_idx 
        self.idx_to_char = idx_to_char  
        self.vocab_size = vocab 
        self.n_h = n_h 
        self.seq_len = seq_len  
        self.epochs = epochs  
        self.lr = learning_rate  
        self.beta1 = beta1  
        self.beta2 = beta2  

        # -----initialise weights and biases----- #
        self.params = {}
        std = (1.0 / np.sqrt(self.vocab_size + self.n_h))  # Xavier initialisation

        # forget gate
        self.params["Wf"] = np.random.randn(self.n_h, self.n_h + self.vocab_size) * std
        self.params["bf"] = np.ones((self.n_h, 1))
        # input gate
        self.params["Wi"] = np.random.randn(self.n_h, self.n_h + self.vocab_size) * std
        self.params["bi"] = np.zeros((self.n_h, 1))
        # cell gate
        self.params["Wc"] = np.random.randn(self.n_h, self.n_h + self.vocab_size) * std
        self.params["bc"] = np.zeros((self.n_h, 1))
        # output gate
        self.params["Wo"] = np.random.randn(self.n_h, self.n_h + self.vocab_size) * std
        self.params["bo"] = np.zeros((self.n_h, 1))
        # output
        self.params["Wv"] = np.random.randn(self.vocab_size, self.n_h) * \
                            (1.0 / np.sqrt(self.vocab_size))
        self.params["bv"] = np.zeros((self.vocab_size, 1))

        # -----initialise gradients and Adam parameters----- #
        self.grads = {}
        self.adam_params = {}

        for key in self.params:
            self.grads["d" + key] = np.zeros_like(self.params[key])
            self.adam_params["m" + key] = np.zeros_like(self.params[key])
            self.adam_params["v" + key] = np.zeros_like(self.params[key])

        self.smooth_loss = -np.log(1.0 / self.vocab_size) * self.seq_len
        return

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def softmax(self, x):
        e_x = np.exp(x - np.max(x))  # max(x) subtracted for numerical stability
        return e_x / np.sum(e_x)

    def clip_grads(self):
        for key in self.grads:
            np.clip(self.grads[key], -5, 5, out=self.grads[key])
        return

    def reset_grads(self):
        for key in self.grads:
            self.grads[key].fill(0)
        return

    def update_params(self, batch_num):
        for key in self.params:
            self.adam_params["m" + key] = self.adam_params["m" + key] * self.beta1 + \
                                          (1 - self.beta1) * self.grads["d" + key]
            self.adam_params["v" + key] = self.adam_params["v" + key] * self.beta2 + \
                                          (1 - self.beta2) * self.grads["d" + key] ** 2

            m_correlated = self.adam_params["m" + key] / (1 - self.beta1 ** batch_num)
            v_correlated = self.adam_params["v" + key] / (1 - self.beta2 ** batch_num)
            self.params[key] -= self.lr * m_correlated / (np.sqrt(v_correlated) + 1e-8)
        return

    def forward_step(self, x, h_prev, c_prev):
        z = np.row_stack((h_prev, x))

        f = self.sigmoid(np.dot(self.params["Wf"], z) + self.params["bf"])
        i = self.sigmoid(np.dot(self.params["Wi"], z) + self.params["bi"])
        c_bar = np.tanh(np.dot(self.params["Wc"], z) + self.params["bc"])

        c = f * c_prev + i * c_bar
        o = self.sigmoid(np.dot(self.params["Wo"], z) + self.params["bo"])
        h = o * np.tanh(c)

        v = np.dot(self.params["Wv"], h) + self.params["bv"]
        y_hat = self.softmax(v)
        return y_hat, v, h, o, c, c_bar, i, f, z

    def backward_step(self, y, y_hat, dh_next, dc_next, c_prev, z, f, i, c_bar, c, o, h):
        dv = np.copy(y_hat)
        dv[y] -= 1  # yhat - y

        self.grads["dWv"] += np.dot(dv, h.T)
        self.grads["dbv"] += dv

        dh = np.dot(self.params["Wv"].T, dv)
        dh += dh_next

        do = dh * np.tanh(c)
        da_o =         do * o * (1 - o)
        self.grads["dWo"] += np.dot(da_o, z.T)
        self.grads["dbo"] += da_o

        dc = dh * o * (1 - np.tanh(c) ** 2)
        dc += dc_next

        dc_bar = dc * i
        da_c = dc_bar * (1 - c_bar ** 2)
        self.grads["dWc"] += np.dot(da_c, z.T)
        self.grads["dbc"] += da_c

        di = dc * c_bar
        da_i = di * i * (1 - i)
        self.grads["dWi"] += np.dot(da_i, z.T)
        self.grads["dbi"] += da_i

        df = dc * c_prev
        da_f = df * f * (1 - f)
        self.grads["dWf"] += np.dot(da_f, z.T)
        self.grads["dbf"] += da_f

        dz = (np.dot(self.params["Wf"].T, da_f)
              + np.dot(self.params["Wi"].T, da_i)
              + np.dot(self.params["Wc"].T, da_c)
              + np.dot(self.params["Wo"].T, da_o))

        dh_prev = dz[:self.n_h, :]
        dc_prev = f * dc
        return dh_prev, dc_prev

    def forward_backward(self, x_batch, y_batch, h_prev, c_prev):
        x, z = {}, {}
        f, i, c_bar, c, o = {}, {}, {}, {}, {}
        y_hat, v, h = {}, {}, {}

        # Values at t= - 1
        h[-1] = h_prev
        c[-1] = c_prev

        loss = 0
        for t in range(self.seq_len):
            x[t] = np.zeros((self.vocab_size, 1))
            x[t][x_batch[t]] = 1

            y_hat[t], v[t], h[t], o[t], c[t], c_bar[t], i[t], f[t], z[t] = \
                self.forward_step(x[t], h[t - 1], c[t - 1])

            loss += -np.log(y_hat[t][y_batch[t], 0])

        self.reset_grads()

        dh_next = np.zeros_like(h[0])
        dc_next = np.zeros_like(c[0])

        for t in reversed(range(self.seq_len)):
            dh_next, dc_next = self.backward_step(y_batch[t], y_hat[t], dh_next,
                                                  dc_next, c[t - 1], z[t], f[t], i[t],
                                                  c_bar[t], c[t], o[t], h[t])
        return loss, h[self.seq_len - 1], c[self.seq_len - 1]
    
    def sample(self, h_prev, c_prev, sample_size):
        x = np.zeros((self.vocab_size, 1))
        h = h_prev
        c = c_prev
        sample_string = ""

        for t in range(sample_size):
            y_hat, _, h, _, c, _, _, _, _ = self.forward_step(x, h, c)
            idx = np.random.choice(range(self.vocab_size), p=y_hat.ravel())
            x = np.zeros((self.vocab_size, 1))
            x[idx] = 1
            char = self.idx_to_char[idx]
            sample_string += char
        return sample_string

    def train(self, X, verbose=True):
        losses = []  

        num_batches = len(X) // self.seq_len
        X_trimmed = X[: num_batches * self.seq_len]  # trim input to have full sequences

        for epoch in range(self.epochs):
            h_prev = np.zeros((self.n_h, 1))
            c_prev = np.zeros((self.n_h, 1))

            for j in range(0, len(X_trimmed) - self.seq_len, self.seq_len):
                # prepare batches
                x_batch = [self.char_to_idx[ch] for ch in X_trimmed[j: j + self.seq_len]]
                y_batch = [self.char_to_idx[ch] for ch in X_trimmed[j + 1: j + self.seq_len + 1]]

                loss, h_prev, c_prev = self.forward_backward(x_batch, y_batch, h_prev, c_prev)

                # smooth out loss and store in list
                self.smooth_loss = self.smooth_loss * 0.999 + loss * 0.001
                losses.append(self.smooth_loss)

                self.clip_grads()

                batch_num = epoch * self.epochs + j / self.seq_len + 1
                self.update_params(batch_num)

                # print out loss and sample string
                if verbose:
                    if j % 400000 == 0:
                        print('Epoch:', epoch+1, '\tBatch:', j, "-", j + self.seq_len,
                              '\tLoss:', round(self.smooth_loss, 2))
                        s = self.sample(h_prev, c_prev, sample_size=50)
                        print(s, "\n")
        return losses, self.params
    
    def test(self, sample_size=100):
        h_prev = np.zeros((self.n_h, 1))
        c_prev = np.zeros((self.n_h, 1))
        return self.sample(h_prev, c_prev, sample_size)

In [8]:
# creating the encoding decoding dictionaries
char_to_idx = {w: i for i, w in enumerate(chars)}
idx_to_char = {i: w for i, w in enumerate(chars)}
lstmParameters = {
        'epochs': 10,
        'char_to_idx': char_to_idx,
        'idx_to_char': idx_to_char,
        'vocab': vocab,
        'learning_rate': 1e-2,
    }

## **LSTM Architecture**

In LSTMs, instead of just a simple network with a single activation function, we have multiple components, giving power to the network to forget and remember information.


<img src="images/lstm.gif"/>

Here's a breakdown of the architecture:

**1. Core Component: LSTM Cell**
* The core of rnnLSTM is the LSTM cell, which processes sequences of data. 
* It manages a cell state that can remember information for long periods.
* This cell state is updated through gates that control the flow of information.

**2. Forget Gate (dWf, dbf)**
* This gate decides what information to forget from the previous cell state (c_prev).
* It considers both the previous hidden state (h_prev) and the current input (x_t) to make this decision.
* The forget gate uses a sigmoid activation function to output values between 0 and 1. 
    * A value close to 1 indicates keeping the information, while a value close to 0 indicates forgetting it entirely.

**3. Input Gate (dWi, dbi)**
* This gate controls which new information gets added to the cell state.
* It has two parts:
    * A sigmoid layer (da_i) that decides which values to update.
    * A tanh layer (c_bar) that creates candidate values for the new information.
* The forget gate's output and the candidate values are combined to determine what information is actually added to the cell state.

**4. Cell State (dc, dWc, dbc)**
* This is the core memory unit of the LSTM cell. 
* It holds the information that is passed through the sequence.
* The cell state is updated by combining the forget gate's output with the previous cell state and the input gate's contribution.

**5. Output Gate (dWo, dbo)**
* This gate controls what information from the cell state is passed on to the next time step's hidden state (h_t).
* It uses a sigmoid layer (da_o) to determine which parts of the cell state are relevant for the output.
* The cell state is then passed through a tanh function and multiplied by the output gate's output to create the final hidden state.

**6. Output Layer (dWv, dbv)**
* The final output of the rnLSTM unit (y_hat) is generated by taking the current hidden state (h_t) and projecting it to the vocabulary size using a weights matrix (Wv) and a bias vector (bv).
* A softmax function is applied to the output to give a probability distribution over the vocabulary, indicating the likelihood of each character being the next character in the sequence. 

In [9]:
LSTM = rnnLSTM(**lstmParameters)
lossLSTM, params = LSTM.train(text)

Epoch: 1 	Batch: 0 - 25 	Loss: 86.64
ws qgsabipm !mnsyaswnoqrvkeqg3 dqyfpkeuot43?c! ccm 

Epoch: 2 	Batch: 0 - 25 	Loss: 51.79
eh thi  wrscuble uncle vernon thes with of letter  

Epoch: 3 	Batch: 0 - 25 	Loss: 40.99
ripbed on the timing touted it it for was ennt lor 

Epoch: 4 	Batch: 0 - 25 	Loss: 37.03
en dudley .in his smelting .boary bled goodly .dud 

Epoch: 5 	Batch: 0 - 25 	Loss: 35.03
op the some and mafe mo.then .young schoced .sked  

Epoch: 6 	Batch: 0 - 25 	Loss: 33.77
ull notes brown uncle vernon his each bit fach to  

Epoch: 7 	Batch: 0 - 25 	Loss: 32.97
end foring to dorm a tant knew and he miders .harr 

Epoch: 8 	Batch: 0 - 25 	Loss: 32.28
re could of that it was spoken his raused down !st 

Epoch: 9 	Batch: 0 - 25 	Loss: 31.86
rould .this marge the bay out !she seppees which w 

Epoch: 10 	Batch: 0 - 25 	Loss: 31.56
rived it dadgy bedreas .they was pouming ive strai 



In [10]:
LSTM.test(100)

'k tos about .uncle vernon ive she set at on the wink yough he saad speering baysed the time as awnin'

In [11]:
RNN.test(100, 10)

'toout t anongisple .ne fbeslele waiton bopufomotowey ozanatof my s sryinon t ok f dray f ns s lld te'

# References

1. [SimpliLearn Deep Learning Tutirials](https://www.simplilearn.com/tutorials/deep-learning-tutorial/rnn)
2. [Introduction to Long Short-Term Memory (LSTM) - Archit Saxena](https://medium.com/analytics-vidhya/introduction-to-long-short-term-memory-lstm-a8052cd0d4cd)
